**Table of contents**<a id='toc0_'></a>    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Libraries](#toc0_)

In [84]:
import os
import numpy as np
import time
os.chdir(os.environ.get('PROJECT_PATH'))
from secnlp.ml_logic import data as d
from secnlp.ml_logic import parsing as p
from secnlp.ml_logic import preprocessing as pre
import secnlp.ml_logic.parsing
from secnlp import utils as u
from secnlp.params import *
import pandas as pd
import importlib
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer


# <a id='toc2_'></a>[Loading 10 text filings per type](#toc0_)

In [4]:
# load whole filings table
df = u.read_data_from_bq(credentials = SERVICE_ACCOUNT, gcp_project = PROJECT, bq_dataset = DATASET_ID, table = FILINGS_10KQ_TABLE_ID)
df['date_filed'] = pd.to_datetime(df['date_filed'])

In [5]:
# sample 10 "Management & Discussion Analysis" sections of 10-K filings
importlib.reload(secnlp.ml_logic.parsing)
filing_sample_10k = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-K')].sample(10)
filing_sample_10k['raw_filing'] = filing_sample_10k['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))
filing_sample_10k['mda'] = filing_sample_10k['raw_filing'].apply(lambda x: p.parse_10k_filing_items(x, item = '7'))


Unable to locate Item 7
Unable to locate Item 7
Unable to locate Item 7


In [6]:
# sample 10 "Management & Discussion Analysis" sections of 10-Q filings
filing_sample_10q = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-Q')].sample(10)
filing_sample_10q['raw_filing'] = filing_sample_10q['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))
filing_sample_10q['mda'] = filing_sample_10q['raw_filing'].apply(lambda x: p.parse_10q_filing_items(x, item = '2'))

Unable to locate Item 2
Unable to locate Item 2
Unable to locate Item 2


# Preprocessing

 ## Preprocessing

In [46]:
# Cleaning
text = p.cleaning(filing_sample_10k['mda'].iloc[1])
# Tokenizing
tokenized = word_tokenize(text)
# Lemmatizing
verb_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "v") for word in tokenized]
noun_verb_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "n") for word in verb_lemmatized]
# Vectorizing
vectorizer = CountVectorizer(stop_words=None,ngram_range = (2,2))
X_bow = vectorizer.fit_transform([" ".join(noun_verb_lemmatized)])

In [87]:
# Apply full preprocessing pipeline to the data
print(pre.pipeline_without_stop_words)
X_bow = pre.pipeline_without_stop_words.fit_transform([filing_sample_10k['mda'].iloc[1]])
display(pd.DataFrame(X_bow.toarray()))

Pipeline(steps=[('cleaning',
                 FunctionTransformer(func=<function cleaning at 0x142e6cb80>)),
                ('vectorizing',
                 CountVectorizer(ngram_range=(2, 2),
                                 tokenizer=<secnlp.ml_logic.preprocessing.LemmaTokenizer object at 0x147a8d690>))])


/Users/Debora/.pyenv/versions/3.10.6/envs/nlpsec/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,506,507,508,509,510
0,8,68,114,46,14,4,6,11,14,5,...,8,45,2,14,2,2,6,6,6,25
